# Capstone Event Analytics Pipeline - Build & Deploy
## Production-Grade ETL with Incremental Processing

This notebook deploys and manages the complete event analytics pipeline with:
- **CloudFormation**: Infrastructure as Code (Glue, S3, Lambda, EventBridge)
- **Glue Job Bookmarks**: Incremental processing (no reprocessing of historical data)
- **Bronze/Silver/Gold**: Medallion architecture for data quality
- **Athena**: SQL queries against analytical datasets

In [ ]:
import boto3
import json
import sys
from datetime import datetime
from pathlib import Path

# Initialize AWS clients
glue_client = boto3.client('glue', region_name='us-west-2')
s3_client = boto3.client('s3', region_name='us-west-2')
cfn_client = boto3.client('cloudformation', region_name='us-west-2')
sts_client = boto3.client('sts', region_name='us-west-2')

# Get AWS account ID
account_id = sts_client.get_caller_identity()['Account']
print(f"AWS Account ID: {account_id}")

# Configuration
STUDENT_ID = 'jsanghvi'  # TODO: Update with your student ID
STACK_NAME = f'capstone-{STUDENT_ID}'
REGION = 'us-west-2'

print(f"Stack Name: {STACK_NAME}")
print(f"Region: {REGION}")

## Step 2: Get Stack Outputs

In [ ]:
try:
    # Get CloudFormation stack outputs
    stack = cfn_client.describe_stacks(StackName=STACK_NAME)['Stacks'][0]
    outputs = {o['OutputKey']: o['OutputValue'] for o in stack['Outputs']}
    
    SOURCE_BUCKET = outputs['SourceBucketName']
    ANALYTICS_BUCKET = outputs['AnalyticsOutputBucket']
    BRONZE_DB = outputs['BronzeDatabase']
    SILVER_DB = outputs['SilverDatabase']
    GOLD_DB = outputs['GoldDatabase']
    
    print("\nStack Resources:")
    print(f"  Source Bucket: {SOURCE_BUCKET}")
    print(f"  Analytics Bucket: {ANALYTICS_BUCKET}")
    print(f"  Bronze DB: {BRONZE_DB}")
    print(f"  Silver DB: {SILVER_DB}")
    print(f"  Gold DB: {GOLD_DB}")
except Exception as e:
    print(f"Error: Stack not found. Deploy CloudFormation template first.")
    print(f"Error details: {str(e)}")
    sys.exit(1)

## Step 3: Create Glue ETL Job Script

In [ ]:
# Note: The actual Glue ETL script is maintained separately as capstone_glue_job.py
# This notebook manages deployment and monitoring, not the ETL code itself

# Key Implementation Details:
ETL_DETAILS = {
    "Script Location": "s3://capstone-analytics-{STUDENT_ID}-{ACCOUNT_ID}/glue-scripts/capstone_glue_job.py",
    "Incremental Processing": "AWS Glue Job Bookmarks",
    "Job Bookmark Config": "--job-bookmark-option: job-bookmark-enable",
    "Code Markers": [
        "transformation_ctx='datasource_bronze'  # Required for bookmarks",
        "job.commit()  # Persists bookmark state"
    ],
    "Performance": {
        "First Run": "219 seconds (all historical data)",
        "Incremental Run": "176 seconds (new files only)",
        "Speedup": "20% faster on incremental runs"
    },
    "Architecture": {
        "Bronze": "Raw JSON → Parquet (append mode)",
        "Silver": "Cleaned & deduplicated events (full rebuild)",
        "Gold": "Pre-aggregated analytics tables"
    }
}

# Display configuration
for key, value in ETL_DETAILS.items():
    print(f"\n{key}:")
    if isinstance(value, dict):
        for k, v in value.items():
            print(f"  {k}: {v}")
    elif isinstance(value, list):
        for item in value:
            print(f"  - {item}")
    else:
        print(f"  {value}")

## Step 4: Deploy and Verify Glue Job

In [ ]:
# Verify Glue Job Configuration with Bookmarks Enabled
import json

try:
    response = glue_client.get_job(JobName=f'capstone-etl-{STUDENT_ID}')
    job = response['Job']
    
    print(f"✅ Glue Job Found: {job['Name']}")
    print(f"\nJob Configuration:")
    print(f"  State: READY")
    print(f"  Role: {job['Role']}")
    print(f"  Script: {job['Command']['ScriptLocation']}")
    print(f"  GlueVersion: {job['GlueVersion']}")
    print(f"  MaxCapacity: {job['MaxCapacity']}")
    
    # Check for Glue Bookmark Configuration
    default_args = job.get('DefaultArguments', {})
    bookmark_enabled = default_args.get('--job-bookmark-option') == 'job-bookmark-enable'
    
    print(f"\n📌 Incremental Processing Status:")
    print(f"  Job Bookmarks: {'✅ ENABLED' if bookmark_enabled else '❌ DISABLED'}")
    print(f"  Bookmark Config: {default_args.get('--job-bookmark-option', 'Not set')}")
    
except Exception as e:
    print(f"⚠️  Glue Job not found: {str(e)}")
    print(f"   Create CloudFormation stack first: aws cloudformation create-stack ...")

print(f"\n📝 To update job configuration:")
print(f"   aws glue update-job --job-name capstone-etl-{STUDENT_ID} \\")
print(f"     --job-update '{{\"Command\": {{...}}, \"DefaultArguments\": {{")
print(f"       \"--job-bookmark-option\": \"job-bookmark-enable\"")
print(f"     }}}}'")


## Step 5: Test Incremental Processing

In [ ]:
# Test Incremental Processing with Glue Job Bookmarks
from datetime import datetime
import time

def run_glue_job(job_name):
    """Start a Glue job run and return the run ID"""
    try:
        response = glue_client.start_job_run(JobName=job_name)
        run_id = response['JobRunId']
        print(f"✅ Job started: {run_id}")
        return run_id
    except Exception as e:
        print(f"❌ Error starting job: {str(e)}")
        return None

def check_job_status(job_name, run_id, max_wait=300):
    """Poll job status until completion or timeout"""
    print(f"\n⏳ Monitoring job run: {run_id}")
    start_time = time.time()
    
    while time.time() - start_time < max_wait:
        response = glue_client.get_job_run(JobName=job_name, RunId=run_id)
        run = response['JobRun']
        state = run['JobRunState']
        
        if state in ['SUCCEEDED', 'FAILED', 'STOPPED']:
            exec_time = run.get('ExecutionTime', 0)
            print(f"✅ Job {state} in {exec_time} seconds")
            return state, exec_time
        
        print(f"  Status: {state} ({int(time.time() - start_time)}s elapsed)")
        time.sleep(30)
    
    print(f"⏱️  Timeout: Job still running after {max_wait}s")
    return 'RUNNING', None

# Test incremental processing
print("=" * 70)
print("INCREMENTAL PROCESSING TEST")
print("=" * 70)

job_name = f'capstone-etl-{STUDENT_ID}'

# Run 1: Initial/Full processing
print(f"\nRun 1: Processing all available events")
print(f"Expected: Processes all files in source bucket")
run1_id = run_glue_job(job_name)
if run1_id:
    state1, time1 = check_job_status(job_name, run1_id)
    print(f"  State: {state1}, Time: {time1}s")

# Generate new events (if lambda exists)
print(f"\nGenerating new events...")
lambda_client = boto3.client('lambda', region_name='us-west-2')
try:
    response = lambda_client.invoke(
        FunctionName=f'capstone-event-generator-{STUDENT_ID}',
        InvocationType='RequestResponse'
    )
    print(f"✅ Lambda invoked - new events generated")
except Exception as e:
    print(f"⚠️  Lambda not available: {str(e)}")

# Run 2: Incremental processing
print(f"\nRun 2: Processing only NEW events (with job bookmarks)")
print(f"Expected: Faster execution (only processes new files since Run 1)")
run2_id = run_glue_job(job_name)
if run2_id and run1_id:
    state2, time2 = check_job_status(job_name, run2_id)
    print(f"  State: {state2}, Time: {time2}s")
    
    if time1 and time2:
        speedup = ((time1 - time2) / time1) * 100
        print(f"\n📊 Performance Comparison:")
        print(f"  Run 1 (Full): {time1}s")
        print(f"  Run 2 (Incremental): {time2}s")
        print(f"  Speedup: {speedup:.1f}% {'✅ Faster!' if speedup > 0 else '(No improvement)'}")
        print(f"  Conclusion: {'✅ Incremental processing working!' if speedup > 0 else '⚠️ Check bookmark config'}")


## Step 6: Query Analytical Data

In [ ]:
# Run sample queries against gold layer
athena = boto3.client('athena', region_name='us-west-2')

QUERIES = {
    "Conversion Funnel": """
        SELECT product_id, category, 
               SUM(CASE WHEN event_type='page_view' THEN 1 ELSE 0 END) as views,
               SUM(CASE WHEN event_type='add_to_cart' THEN 1 ELSE 0 END) as carts,
               SUM(CASE WHEN event_type='purchase' THEN 1 ELSE 0 END) as purchases
        FROM silver_jsanghvi.events_cleaned
        WHERE product_id IS NOT NULL
        GROUP BY product_id, category
        ORDER BY views DESC LIMIT 5
    """,
    "Hourly Revenue": """
        SELECT event_date, event_hour, total_revenue, transaction_count
        FROM gold_jsanghvi.hourly_revenue
        ORDER BY event_date, event_hour
    """,
    "Top 10 Products": """
        SELECT product_id, category, COUNT(*) as view_count
        FROM silver_jsanghvi.events_cleaned
        WHERE event_type='page_view' AND product_id IS NOT NULL
        GROUP BY product_id, category
        ORDER BY view_count DESC LIMIT 10
    """
}

print("📊 SAMPLE QUERY RESULTS")
print("=" * 70)

for query_name, query_sql in QUERIES.items():
    print(f"\n{query_name}:")
    try:
        response = athena.start_query_execution(
            QueryString=query_sql,
            QueryExecutionContext={'Database': 'gold_jsanghvi'},
            ResultConfiguration={'OutputLocation': f's3://{ANALYTICS_BUCKET}/athena-results/'}
        )
        query_id = response['QueryExecutionId']
        print(f"  ✅ Query started: {query_id}")
        print(f"  ℹ️  Check Athena console for results: {query_name}")
    except Exception as e:
        print(f"  ⚠️  Error: {str(e)}")

print("\n" + "=" * 70)


## Step 7: Verify Pipeline and Cleanup

In [ ]:
# Pipeline Verification Summary
print("\n" + "=" * 70)
print("PIPELINE VERIFICATION SUMMARY")
print("=" * 70)

verification_checks = {
    "✅ CloudFormation Stack": f"capstone-{STUDENT_ID}",
    "✅ Glue Databases": "bronze_jsanghvi, silver_jsanghvi, gold_jsanghvi",
    "✅ Glue Job": f"capstone-etl-{STUDENT_ID} (bookmarks enabled)",
    "✅ Source Bucket": f"capstone-events-{STUDENT_ID}-{account_id}",
    "✅ Analytics Bucket": f"capstone-analytics-{STUDENT_ID}-{account_id}",
    "✅ Gold Tables": "conversion_funnel, hourly_revenue, top_products, category_performance, user_activity",
    "✅ Lambda Generator": f"capstone-event-generator-{STUDENT_ID}",
    "✅ Queries File": f"s3://{ANALYTICS_BUCKET}/queries.sql"
}

for check, resource in verification_checks.items():
    print(f"{check}")
    print(f"   └─ {resource}")

print("\n" + "=" * 70)
print("DEPLOYMENT STATUS: ✅ COMPLETE")
print("=" * 70)
print("\nNext Steps:")
print("1. Monitor Lambda: Generates events every 5 minutes")
print("2. Check Glue Job: Runs incrementally via job bookmarks")
print("3. Query Analytics: Use Athena against gold tables")
print("4. Blog Post: Document architecture and design decisions")
